# RDF2TG 

this notebook introduces how TigerGraph property graph database system works towards support RDF data using LDBC_SPB benchmark.

In [1]:
# Python library
import os
from datetime import datetime
# Graph stuff
import rdflib 
from rdflib import Graph, OWL, RDF, RDFS
# Tigergraph library
import pyTigerGraph as tg
# pandas process dataset
import pandas as pd
import csv

In [2]:
##### Logging
log_output = open("./data/output/rdf2tg_ldbc_spb.log","a+")

def log(msg,tab_level=0,output=log_output):
    timestamp = datetime.now()
    tabs = "\t" * (tab_level+1)
    message = f"{timestamp}{tabs}{msg}"
    print(message)
    output.write(f"{message}\n")

In [3]:
#### Paths
# The directory hosting all input data
data_path = "./data/input/"
# The directory hosting all output data
output_path = "./data/output/"

## Download and preprocess data 

* you can run sparql queries below on your triple store via a sparql endpoint, e.g., installing ontotext graphdb on your local machine/server and run the sparql queries via the endpoint.

* you can directly run the query using the triple store UI and download it directly.

* you can also utilize rdflib python library to load the turtle file and run SPARQL queries to convert the ttl file to csv format.

* for the convenience, we have downloaded the rdf dump and also converted it into two csv files: datatypeproperty.csv and objectproperty.csv, which can be directly mapped and loaded to TigerGraph database. The data files for the ldbc_spb dataset are too large to include in github repo.please download from https://drive.google.com/drive/folders/1FtHs0IK8oYsKY_MDl8RYcMTdKP2kCbx_?usp=sharing and save the data files in the ./data/input folder.

In [4]:
# g = Graph()

# rdf_dump = os.path.join(data_path, "ldbc-spb-32M.ttl")

# g.parse(rdf_dump, format="ttl")

# log("running sparql to get all triple with object properties")

# objectPropertyTriples = """
#     SELECT ?s ?p ?o (MD5(concat(str(?s), str(?p), str(?o))) AS ?pid)
#     WHERE {
#         ?s ?p ?o .
#     filter (!isLiteral(?o)) .
#     }    
# """

# objectproperty = g.query(objectPropertyTriples)

# log("running successfully")

# log("running sparql to get all triple with datatype property properties")

# datatypePropertyTriples = """
#     SELECT ?s ?p ?o (datatype(?o) as ?d) (lang(?o) as ?l) 
#     (MD5(concat(str(?s), str(?p), str(?o), lang(?o))) AS ?pid) 
#     (MD5(concat(str(?o), lang(?o))) AS ?vid)   
#     WHERE {
#         ?s ?p ?o .
#     filter isLiteral(?o) .
#     }    
# """

# datatypeproperty = g.query(datatypePropertyTriples)

# log("running successfully")

# datatype_file = os.path.join(data_path, "datatypeproperty.csv")
# object_file = os.path.join(data_path, "objectproperty.csv")

# with open(datatype_file, "w") as f:
#     writer = csv.DictWriter(f, fieldnames=[str(v) for v in datatypeproperty.vars])
#     writer.writeheader()
#     for binding in datatypeproperty.bindings:
#         writer.writerow({str(k): str(v) for k, v in binding.items()})
        
# with open(object_file, "w") as f1:
#     writer = csv.DictWriter(f1, fieldnames=[str(v) for v in objectproperty.vars])
#     writer.writeheader()
#     for binding in objectproperty.bindings:
#         writer.writerow({str(k): str(v) for k, v in binding.items()})

## Connect to tigergraph cluster

In [5]:
#### Tigergraph Solution Connection
log("connecting to graph cluster.")

# Connection parameters
# Configure to your solution
hostname = "xxxxxxxxxxxxxxx"
username = "xxxxxxx"
password = "xxxxxxx"

conn = tg.TigerGraphConnection(host=hostname, username=username, password=password)

log("successfully connect to cluster")

2022-11-21 14:18:48.821147	connecting to graph cluster.
2022-11-21 14:18:49.449086	successfully connect to cluster


### Create a graph

Next, use the CREATE GRAPH command to create a graph named ldbc_spb.

In [6]:
results = conn.gsql(
'''
CREATE GRAPH ldbc_spb ()
'''
)

log(results)

2022-11-21 14:19:12.061495	The graph ldbc_spb is created.


### Connect to ldbc_spb graph

In [7]:
conn.graphname="ldbc_spb"

conn.getToken(conn.createSecret())
    
log("successfully connect to graph ldbc_spb")

2022-11-21 14:19:16.031006	successfully connect to graph ldbc_spb


### Create vertices and edges

In [8]:
#### Create graph schema in tigergraph 
# create nodes and edges 
log("creating ldbc_spb schema in Tigergraph.")

results = conn.gsql(
    '''
    USE GRAPH ldbc_spb
    
    CREATE SCHEMA_CHANGE JOB ldbc_spb_schema FOR GRAPH ldbc_spb {
        ADD VERTEX ClassInstance (primary_id id STRING, uri STRING);
        ADD VERTEX ObjectPropertyInstance (primary_id id STRING, uri STRING);
        ADD VERTEX DatatypePropertyInstance (primary_id id STRING, uri STRING);
        ADD VERTEX ValueInstance (primary_id id STRING, value STRING, datatype STRING, langTag STRING);

        ADD DIRECTED EDGE hasDatatypePropertyInstance (FROM ClassInstance, TO DatatypePropertyInstance) WITH REVERSE_EDGE="reverse_hasDatatypePropertyInstance";
        ADD DIRECTED EDGE hasObjectPropertyInstance (FROM ClassInstance, TO ObjectPropertyInstance) WITH REVERSE_EDGE="reverse_hasObjectPropertyInstance";
        ADD DIRECTED EDGE hasObjectInstance (FROM ObjectPropertyInstance, TO ClassInstance) WITH REVERSE_EDGE="reverse_hasObjectInstance";
        ADD DIRECTED EDGE hasValueInstance (FROM DatatypePropertyInstance, TO ValueInstance) WITH REVERSE_EDGE="reverse_hasValueInstance";
    }
    
    RUN SCHEMA_CHANGE JOB ldbc_spb_schema
    '''
)

log(results)

2022-11-21 14:19:16.047749	creating ldbc_spb schema in Tigergraph.
2022-11-21 14:19:23.835249	Using graph 'ldbc_spb'
Successfully created schema change jobs: [ldbc_spb_schema].

Current graph version 0
Trying to add vertex ClassInstance.
Trying to add vertex ObjectPropertyInstance.
Trying to add vertex DatatypePropertyInstance.
Trying to add vertex ValueInstance.
Trying to add edge hasDatatypePropertyInstance.
Trying to add edge reverse_hasDatatypePropertyInstance.
Trying to add edge hasObjectPropertyInstance.
Trying to add edge reverse_hasObjectPropertyInstance.
Trying to add edge hasObjectInstance.
Trying to add edge reverse_hasObjectInstance.
Trying to add edge hasValueInstance.
Trying to add edge reverse_hasValueInstance.
Kick off job ldbc_spb_schema

Graph ldbc_spb update to new version 1
The job ldbc_spb_schema completes in 6.869 seconds!


### Create index on vertex and edge properties

In [9]:
log("adding index in ldbc_spb schema in Tigergraph.")

results = conn.gsql('''
    USE GRAPH ldbc_spb

    CREATE SCHEMA_CHANGE JOB attribute_index1 {
        ALTER VERTEX ClassInstance ADD INDEX ClassInstance_uri_index ON (uri);
        ALTER VERTEX ObjectPropertyInstance ADD INDEX ObjectPropertyInstance_uri_index ON (uri);
        ALTER VERTEX DatatypePropertyInstance ADD INDEX DatatypePropertyInstance_uri_index ON (uri);
        ALTER VERTEX ValueInstance ADD INDEX ValueInstance_value_index ON (value);
        ALTER VERTEX ValueInstance ADD INDEX ValueInstance_datatype_index ON (datatype);
        ALTER VERTEX ValueInstance ADD INDEX ValueInstance_langTag_index ON (langTag);
    }
    
    RUN SCHEMA_CHANGE JOB attribute_index1
    '''
)

log(results)

2022-11-21 14:19:23.851811	adding index in ldbc_spb schema in Tigergraph.
2022-11-21 14:19:31.928730	Using graph 'ldbc_spb'
Successfully created schema change jobs: [attribute_index1].

Current graph version 1
Kick off job attribute_index1

Graph ldbc_spb update to new version 2
The job attribute_index1 completes in 7.069 seconds!


### Define loading data job to graph ldbc_spb

In [10]:
#### Map data to the schema
log("Maping LDBC SPB data to Tigergraph schema.")

results = conn.gsql('''
    USE GRAPH ldbc_spb
    
    CREATE LOADING JOB load_data FOR GRAPH ldbc_spb {
        DEFINE FILENAME ObjectProperty_file;
        DEFINE FILENAME DatatypeProperty_file;
        
        LOAD ObjectProperty_file TO EDGE hasObjectPropertyInstance VALUES($0, $3) USING SEPARATOR="|", EOL="\n", QUOTE="double";
        LOAD ObjectProperty_file TO EDGE hasObjectInstance VALUES($3, $2) USING SEPARATOR="|", EOL="\n", QUOTE="double";
        LOAD ObjectProperty_file TO VERTEX ClassInstance VALUES($0, $0) USING SEPARATOR="|", EOL="\n", QUOTE="double";
        LOAD ObjectProperty_file TO VERTEX ObjectPropertyInstance VALUES($3, $1) USING SEPARATOR="|", EOL="\n", QUOTE="double";
        LOAD ObjectProperty_file TO VERTEX ClassInstance VALUES($2, $2) USING SEPARATOR="|", EOL="\n", QUOTE="double";

        LOAD DatatypeProperty_file TO EDGE hasDatatypePropertyInstance VALUES($0, $5) USING SEPARATOR="|", EOL="\n", QUOTE="double";
        LOAD DatatypeProperty_file TO EDGE hasValueInstance VALUES($5, $6) USING SEPARATOR="|", EOL="\n", QUOTE="double";
        LOAD DatatypeProperty_file TO VERTEX ClassInstance VALUES($0, $0) USING SEPARATOR="|", EOL="\n", QUOTE="double";
        LOAD DatatypeProperty_file TO VERTEX DatatypePropertyInstance VALUES($5, $1) USING SEPARATOR="|", EOL="\n", QUOTE="double";
        LOAD DatatypeProperty_file TO VERTEX ValueInstance VALUES($6, $2, $3, $4) USING SEPARATOR="|", EOL="\n", QUOTE="double";
    }
    '''
)

log(results)

2022-11-21 14:19:31.945094	Maping LDBC SPB data to Tigergraph schema.
2022-11-21 14:19:34.177805	Using graph 'ldbc_spb'
Successfully created loading jobs: [load_data].


### Run the loading job with files

since the data files are too large to load, we first split them into smaller chunks and upload them. 

In [11]:
datatype_file = os.path.join(data_path, "datatypeproperty.csv")
object_file = os.path.join(data_path, "objectproperty.csv")

for i, chunk in enumerate(pd.read_csv(datatype_file, chunksize=600000, header=None, low_memory=False, sep=',', quotechar='"', doublequote=True)):
    chunk.to_csv(data_path + 'datatype_chunk{}.csv'.format(i), index=False, header=False, sep='|', quotechar='"', doublequote=True)

for i, chunk in enumerate(pd.read_csv(object_file, chunksize=600000, header=None, low_memory=False, sep=',', quotechar='"', doublequote=True)):
    chunk.to_csv(data_path + 'object_chunk{}.csv'.format(i), index=False, header=False, sep='|', quotechar='"', doublequote=True)

### List all file chucks 

In [12]:
#### Load data to the graph

datatype_file_list = []

for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith(".csv") and "datatype_chunk" in file:
            datatype_file_list.append(os.path.join(root, file))

object_file_list = []

for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith(".csv") and "object_chunk" in file:
            object_file_list.append(os.path.join(root, file))

for datatypeFile in datatype_file_list:
    print (datatypeFile)
    
for objectFile in object_file_list:
    print (objectFile)

./data/input/datatype_chunk15.csv
./data/input/datatype_chunk14.csv
./data/input/datatype_chunk16.csv
./data/input/datatype_chunk17.csv
./data/input/datatype_chunk13.csv
./data/input/datatype_chunk12.csv
./data/input/datatype_chunk10.csv
./data/input/datatype_chunk11.csv
./data/input/datatype_chunk4.csv
./data/input/datatype_chunk5.csv
./data/input/datatype_chunk7.csv
./data/input/datatype_chunk6.csv
./data/input/datatype_chunk2.csv
./data/input/datatype_chunk3.csv
./data/input/datatype_chunk1.csv
./data/input/datatype_chunk0.csv
./data/input/datatype_chunk8.csv
./data/input/datatype_chunk9.csv
./data/input/datatype_chunk20.csv
./data/input/datatype_chunk21.csv
./data/input/datatype_chunk23.csv
./data/input/datatype_chunk22.csv
./data/input/datatype_chunk19.csv
./data/input/datatype_chunk24.csv
./data/input/datatype_chunk18.csv
./data/input/object_chunk11.csv
./data/input/object_chunk5.csv
./data/input/object_chunk4.csv
./data/input/object_chunk10.csv
./data/input/object_chunk12.csv
./

### Upload object property files

In [13]:
log("Loading ldbc spb data to Tigergraph schema.")

for objectFile in object_file_list:
    print ("loading file", objectFile)
    print ("-"*100)
    uploadOPFile = conn.runLoadingJobWithFile(filePath=objectFile, fileTag="ObjectProperty_file", jobName="load_data", sep="|", eol="\n", timeout=0, sizeLimit=200000000)
    log (uploadOPFile)
    print ("\n")

2022-11-21 14:21:44.117977	Loading ldbc spb data to Tigergraph schema.
loading file ./data/input/object_chunk11.csv
----------------------------------------------------------------------------------------------------
2022-11-21 14:22:06.096903	[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 600000, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 0, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'ClassInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ClassInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ObjectPropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'in

2022-11-21 14:24:36.176545	[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 600000, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 0, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'ClassInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ClassInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ObjectPropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'edge': [{'typeName': 'hasObjectPropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 

2022-11-21 14:27:04.504331	[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 155379, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 0, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'ClassInstance', 'validObject': 155379, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ClassInstance', 'validObject': 155379, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ObjectPropertyInstance', 'validObject': 155291, 'noIdFound': 88, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'edge': [{'typeName': 'hasObjectPropertyInstance', 'validObject': 155291, 'noIdFound': 88, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0

2022-11-21 14:30:01.611033	[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 600000, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 0, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'ClassInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ClassInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ObjectPropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'edge': [{'typeName': 'hasObjectPropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 

2022-11-21 14:32:49.640759	[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 600000, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 0, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'ClassInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ClassInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ObjectPropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'edge': [{'typeName': 'hasObjectPropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 

### Upload datatype property files

In [14]:
for datatypeFile in datatype_file_list:
    print ("loading file", datatypeFile)
    print ("-"*100)
    uploadDPFile = conn.runLoadingJobWithFile(filePath=datatypeFile, fileTag="DatatypeProperty_file", jobName="load_data", sep="|", eol="\n", timeout=0, sizeLimit=200000000)
    log (uploadDPFile)
    print ("\n")

loading file ./data/input/datatype_chunk15.csv
----------------------------------------------------------------------------------------------------
2022-11-21 14:35:19.196607	[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 599970, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 65, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'ClassInstance', 'validObject': 599970, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'DatatypePropertyInstance', 'validObject': 599970, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ValueInstance', 'validObject': 599969, 'noIdFound': 1, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'edge': [{'typeName': 'hasDatatype

2022-11-21 14:38:40.992053	[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 599906, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 448, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'ClassInstance', 'validObject': 599906, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'DatatypePropertyInstance', 'validObject': 599906, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ValueInstance', 'validObject': 599906, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'edge': [{'typeName': 'hasDatatypePropertyInstance', 'validObject': 599906, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId

2022-11-21 14:42:35.995722	[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 600000, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 0, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'ClassInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'DatatypePropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ValueInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'edge': [{'typeName': 'hasDatatypePropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId':

2022-11-21 14:46:37.549995	[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 599998, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 2, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'ClassInstance', 'validObject': 599998, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'DatatypePropertyInstance', 'validObject': 599998, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ValueInstance', 'validObject': 599966, 'noIdFound': 32, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'edge': [{'typeName': 'hasDatatypePropertyInstance', 'validObject': 599998, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId'

2022-11-21 14:50:19.473750	[{'sourceFileName': 'Online_POST', 'statistics': {'validLine': 600000, 'rejectLine': 0, 'failedConditionLine': 0, 'notEnoughToken': 0, 'invalidJson': 0, 'oversizeToken': 0, 'vertex': [{'typeName': 'ClassInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'DatatypePropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}, {'typeName': 'ValueInstance', 'validObject': 599873, 'noIdFound': 127, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId': 0, 'incorrectFixedBinaryLength': 0}], 'edge': [{'typeName': 'hasDatatypePropertyInstance', 'validObject': 600000, 'noIdFound': 0, 'invalidAttribute': 0, 'invalidVertexType': 0, 'invalidPrimaryId': 0, 'invalidSecondaryId

### Get vertex/edge statistics

In [15]:
vertices = conn.getVertexTypes()
total_count = 0

for vertex in vertices:
    vertex_cnt = conn.getVertexCount(vertex)
    total_count += vertex_cnt
    print("Node count: ({} : {}) ".format(vertex, vertex_cnt))
print("Total node count: ", total_count)

Node count: (ClassInstance : 3946393) 
Node count: (ObjectPropertyInstance : 16955079) 
Node count: (DatatypePropertyInstance : 14820604) 
Node count: (ValueInstance : 5165683) 
Total node count:  40887759


In [16]:
import pprint
edge_count = conn.getEdgeCount()
print("Edges count: total ", sum(edge_count.values()))
pprint.pprint(edge_count) 

Edges count: total  124421814
{'hasDatatypePropertyInstance': 14086033,
 'hasObjectInstance': 16955079,
 'hasObjectPropertyInstance': 16955079,
 'hasValueInstance': 14061041,
 'reverse_hasDatatypePropertyInstance': 14073247,
 'reverse_hasObjectInstance': 16955079,
 'reverse_hasObjectPropertyInstance': 16955079,
 'reverse_hasValueInstance': 14381177}


## UDF installation

In GSQL, users can supplement the language by defining their own query user-defined functions (query UDFs) in C++. Query UDFs can be called in queries and subqueries to perform a set of defined actions and return a value like the built-in functions. 

In this project, a type conversation function str_to_float() is not built-in in TigerGraph DB, which needs to install by following the steps below. 

This page introduces the process to define a query UDF. Once defined, the new functions are added into GSQL automatically the next time GSQL is executed. (https://docs.tigergraph.com/gsql-ref/current/querying/func/query-user-defined-functions)

There are two ways to modify the UDF files to add user-defined functions to GSQL. Both are secured by gadmin and require administrator-level access

### Use GitHub to store UDFs

* Store UDF files in a GitHub repository, and configure GSQL to read from the repository.

    * This is the recommended approach. It also takes the highest precedence. If GSQL is configured to read from GitHub for UDFs and the configurations are valid, UDFs in other sources are ignored. Two hpp files (ExprFunctions.hpp and ExprUtil.hpp) are provided in the ./data/input/udf 
    
Use the gadmin config set command to configure the aforementioned parameters to connect GSQL to the GitHub repository hosting your files.

The following is an example configuration. Remember to run gadmin config apply after changing the parameters. If GSQL is already running, run gadmin restart all to restart GSQL before the UDFs become available.

* gadmin config set GSQL.GithubUserAcessToken xxxxxxxxxxx
* gadmin config set GSQL.GithubRepository repo name
* gadmin config set GSQL.GithubBranch branch name
* gadmin config set GSQL.GithubPath path/to/code
* gadmin config apply

After the parameters are successfully configured, you can access your UDFs in new queries right away.

### Upload UDFs with GSQL PUT command

Run the following command to enable uploading query UDFs through the GSQL PUT command:

* gadmin config set GSQL.UDF.EnablePutExpr true
* gadmin config apply
* gadmin restart gsql

use the PUT command to upload the files you modified.

* GSQL > PUT ExprFunctions FROM </path/to/udf_file.hpp>
* PUT ExprFunctions successfully.
* GSQL > PUT ExprUtil FROM </path/to/utils_file.hpp>
* PUT ExprUtil successfully.

The PUT command automatically uploads the files in all nodes in a cluster and updates all existing files. Once the files are stored, you can call the UDFs in a query the next time GSQL is executed. 

## Create all gsql queries

In [17]:
#### create queries
log("Create GSQL query")

basic_path = data_path + "queries/gsql/basic"

advanced_path = data_path + "queries/gsql/advanced"

def createQuery(file_path):
    with open(file_path, 'r') as f:
        query = f.read()
        results = conn.gsql("USE GRAPH ldbc_spb\n\n" + query)
        log(results)

for file in os.listdir(basic_path):
    if file.endswith(".txt"):
        file_path = f"{basic_path}/{file}"
        createQuery(file_path)
        
for file in os.listdir(advanced_path):
    if file.endswith(".txt"):
        file_path = f"{advanced_path}/{file}"
        createQuery(file_path)

2022-11-21 14:50:33.213024	Create GSQL query
2022-11-21 14:50:34.439395	Successfully created queries: [basic_query3_optimized].
2022-11-21 14:50:35.261386	Successfully created queries: [basic_query11_optimized].
2022-11-21 14:50:36.344281	Successfully created queries: [basic_query10_optimized].
2022-11-21 14:50:37.470488	Successfully created queries: [basic_query2_optimized].
2022-11-21 14:50:39.219104	Successfully created queries: [basic_query1_optimized].
2022-11-21 14:50:40.318508	Successfully created queries: [basic_query5_optimized].
2022-11-21 14:50:41.465190	Successfully created queries: [basic_query4_optimized].
2022-11-21 14:50:42.600186	Warning in query basic_query6_optimized (WARN-5): line 221, col 18
The comparison
'str_to_float(s.@latitude)>=(41.57930219264732-0.2232188735361287)' may lead to
unexpected behavior because it involves equality test between float/double
numeric values. We suggest to do such comparison with an error margin, e.g.
'str_to_float(s.@latitude)>=(41.

### Install all queries 

In [18]:
# Install queries 
log("Installig all queries")

results = conn.gsql("""
    USE GRAPH ldbc_spb
    INSTALL QUERY ALL
""")

log(results)

2022-11-21 14:51:12.406214	Installig all queries
2022-11-21 14:53:04.420861	Using graph 'ldbc_spb'
Start installing queries, about 1 minute ...
advanced_query18_optimized query: curl -X GET 'https://127.0.0.1:9000/query/ldbc_spb/advanced_query18_optimized'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
advanced_query4_optimized query: curl -X GET 'https://127.0.0.1:9000/query/ldbc_spb/advanced_query4_optimized'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
advanced_query24_optimized query: curl -X GET 'https://127.0.0.1:9000/query/ldbc_spb/advanced_query24_optimized'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
advanced_query2_optimized query: curl -X GET 'https://127.0.0.1:9000/query/ldbc_spb/advanced_query2_optimized'. Add -H "Authorization: Bearer TOKEN" if authentication is enabled.
basic_query4_optimized query: curl -X GET 'https://127.0.0.1:9000/query/ldbc_spb/basic_query4_optimized'. Add -H "Authorization: Bearer 

### Run basic queries

In [19]:
#### initialize the parameters and run basic queries

# create a dictionary to store the queryname -> runtime pair
runtimes = {}

# execute 11 basic queries
for i in range(1, 12):
    starttime = datetime.now()
    results = conn.runInstalledQuery("basic_query" + str(i) + "_optimized")
    endtime = datetime.now()
    runtime = endtime - starttime
    runtimes["basic_query" + str(i) + "_optimized"] = runtime.total_seconds()

print("running 11 basic queries successfully")

running 11 basic queries successfully


### Run advanced queries

In [20]:
#### initialize the parameters and run advanced queries

# execute 25 advanced queries
for i in range(1, 26):

    starttime = datetime.now()
    results = conn.runInstalledQuery("advanced_query" + str(i) + "_optimized")
    endtime = datetime.now()  
    runtime = endtime - starttime
    runtimes["advanced_query" + str(i) + "_optimized"] = runtime.total_seconds()

print("running 25 advanced queries successfully")

running 25 advanced queries successfully


### display execution time of all queries

In [21]:
# print out the query performance 
for x, y in runtimes.items():
    print(x, y)

basic_query1_optimized 0.733015
basic_query2_optimized 0.632512
basic_query3_optimized 0.71657
basic_query4_optimized 0.680404
basic_query5_optimized 0.69692
basic_query6_optimized 1.034583
basic_query7_optimized 0.946427
basic_query8_optimized 0.734086
basic_query9_optimized 0.635203
basic_query10_optimized 1.066987
basic_query11_optimized 0.648502
advanced_query1_optimized 0.703529
advanced_query2_optimized 0.641884
advanced_query3_optimized 0.666149
advanced_query4_optimized 0.677813
advanced_query5_optimized 0.69724
advanced_query6_optimized 0.836308
advanced_query7_optimized 0.679417
advanced_query8_optimized 0.654185
advanced_query9_optimized 0.650692
advanced_query10_optimized 0.687349
advanced_query11_optimized 0.637687
advanced_query12_optimized 0.640025
advanced_query13_optimized 1.308295
advanced_query14_optimized 1.236409
advanced_query15_optimized 1.083035
advanced_query16_optimized 0.991538
advanced_query17_optimized 0.933807
advanced_query18_optimized 1.072151
advanced_q